<a href="https://colab.research.google.com/github/JevinKOpenEyes/colab-copies/blob/main/Tagging_and_Summarization%20-%200508%20JK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q auto-gptq transformers tika aspose-words accelerate

In [ ]:
import requests
from tika import parser
from bs4 import BeautifulSoup
from io import StringIO

In [ ]:
# This line gets ssf6.pdf files metadata from the google drive URI
request=requests.get("https://generate-questions.devbyopeneyes.com/api/getFileData/66acaf7a1571ea56e405e9c2")
resp = request.json()

# Getting file metadata
file_path = (resp["data"]["file_path"])
file_name = (resp["data"]["file_name"])
_id = (resp["data"]["_id"])
file_type = (resp["data"]["file_type"])
user_file_name = (resp["data"]["file_name"])
type_of_question = (resp["data"]["type_of_question"])

In [ ]:
if file_type == "pdf" or file_type == "txt":
    Text_pdf = []
    # Read PDF file
    data = parser.from_file(file_path, xmlContent=True)
    xhtml_data = BeautifulSoup(data['content'])
    for i, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
        # Parse PDF data using TIKA (xml/html)
        # It's faster and safer to create a new buffer than truncating it
        # https://stackoverflow.com/questions/4330812/how-do-i-clear-a-stringio-object
        _buffer = StringIO()
        _buffer.write(str(content))
        parsed_content = parser.from_buffer(_buffer.getvalue())

        # Add pages
        if parsed_content['content']!=None:
            text = parsed_content['content'].strip()
            Text_pdf.append(text)

2024-08-05 07:22:28,446 [MainThread  ] [INFO ]  Retrieving https://drive.google.com/u/0/uc?id=1ozCc6YVeBOy5047AeIJpj1WvbOJuuZDr&export=download to /tmp/u-0-uc.
INFO:tika.tika:Retrieving https://drive.google.com/u/0/uc?id=1ozCc6YVeBOy5047AeIJpj1WvbOJuuZDr&export=download to /tmp/u-0-uc.


In [ ]:
combined_text = "\n".join(Text_pdf)

In [ ]:
combined_text

'HB 207  2024 \n\n\n\n\n\n\n\n\n\n\n\nCODING: Words stricken are deletions; words underlined are additions. \n\n\nhb0207-00 \n\n\nPage 1 of 4 \n\n\nF L O R I D A  H O U S E  O F  R E P R E S E N T A T I V E S \n\n\n\n\n\n\n\n\n\n\n\nA bill to be entitled 1 \n\n\nAn act relating to social media protection for minors; 2 \n\n\ncreating s. 501.174, F.S.; providing a definition; 3 \n\n\nrequiring social media platforms to disclose specified 4 \n\n\npolicies and provide specified resources, measures, 5 \n\n\nand disclaimers; authorizing social media platforms to 6 \n\n\npost specified compliance statements on their Internet 7 \n\n\nhomepage or platform user login page; prohibiting 8 \n\n\ncertain schools from using or having an account on 9 \n\n\ncertain social media platforms and from requiring 10 \n\n\nstudents to register, enroll, or participate in social 11 \n\n\nmedia platforms for educational purposes; providing 12 \n\n\napplicability; providing an effective date. 13 \n\n\n 14 \n\n\nBe

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
facebook_model_summary = summarizer(combined_text, max_length=300, min_length=100, do_sample=False)

IndexError: index out of range in self

In [ ]:
merged_summary_facebook = " ".join(item['summary_text'] for item in facebook_model_summary)
print(merged_summary)

NameError: name 'merged_summary' is not defined

In [ ]:
merged_summary_facebook

In [ ]:
from transformers import pipeline

pipe = pipeline(model="facebook/bart-large-mnli")


In [ ]:
classified_tag_facebook = pipe(merged_summary_facebook,
    candidate_labels=["Budget", "Education", "Government", "Data", "State Response","Healthcare","Other", "Privacy", "Military",  "Telecom", "Energy","Transportation","Social", "Labor","AG"],
)

In [ ]:
classified_tag_facebook

In [ ]:
Text_pdf

In [ ]:
from transformers import pipeline
pipe = pipeline("summarization", model="google/pegasus-xsum")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
google_model_summary = pipe(Text_pdf, max_length=295, min_length=100, do_sample=False)

KeyboardInterrupt: 

In [ ]:
merged_summary_google = " ".join(item['summary_text'] for item in google_model_summary)

In [ ]:
merged_summary_google = """HB 207 is a bill to be entitled "An act relating to social media protection for minors" It would require social media platforms to disclose specified policies and provide specified resources, measures, and disclaimers. It would also prohibit schools from using or having an account on social media. Beginning January 1, 2025, a social media platform must disclose the following policies. It must be clearly, concisely, prominently, and clearly written using language suited to the age of users. uses to protect platform users who are under the age of 18. Whether the social media platform collects or sells personal information of platform users. Zip code-based references to local resources for law. This application may be harmful to your mental health. A social media platform that complies with subsections 84, 85, and (3) may post a statement confirming such compliance on 85 its Internet homepage or platform user login page. A public K-12 school may not: (a)  Use or have an account on a social mediaplatform that 88, 89, 90 does not have a statement pursuant to subsection """

In [ ]:
from transformers import pipeline

pipe = pipeline(model="facebook/bart-large-mnli")
classfied_tag_google = pipe(merged_summary_google,
    candidate_labels=["Budget", "Education", "Government", "Data", "State Response","Healthcare","Other", "Privacy", "Military",  "Telecom", "Energy","Transportation","Social", "Labor","AG"],
)

In [ ]:
classfied_tag_google

In [ ]:
from transformers import pipeline
classifier = pipeline('zero-shot-classification', model='roberta-large-mnli')


In [ ]:
sequence_to_classify = "Bharti Airtel, the second largest operator, provided salary hikes in the range of 10-12%. Loss-making Vodafone Idea, with over 9,000 employees, also gave hikes of 8-11%. Reliance Jio Infocomm has yet to announce the hikes for this year."

In [ ]:

candidate_labels = ["Budget", "Education", "Government", "Data", "State Response","Healthcare","Other", "Privacy", "Military",  "Telecom", "Energy","Transportation","Social", "Labor","AG"]
classifier(sequence_to_classify, candidate_labels)
